In [18]:

!pip install boto3
!pip install h2o


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.8/265.8 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.46.0.6-py2.py3-none-any.whl size=265859786 sha256=98575453beae511aa11bd6f2dc9131654d12d795f93205b99f5bca22bbdccda9
  Stored in directory: /root/.cache/pip/wheels/0c/95/eb/b449c73f478dbc4557b80170cf7f2aa6db15862877e9a63536
Successfully built h2o


Importing Libraries

In [19]:
import boto3
import pandas as pd
from io import StringIO
import os
import h2o
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split

## Data Import from S3

Credentials for access

In [23]:

# Initialize AWS session
session = boto3.Session(
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name='us-east-1'
)

s3 = session.client('s3')

Accessing the data from S3

In [24]:
# Step 1: Load the CSV file from S3
bucket = 'newscatayerhs'
key = 'Customer-Churn-Records.csv'

obj = s3.get_object(Bucket=bucket, Key=key)
csv_string = obj['Body'].read().decode('utf-8')

# Step 2: Read the CSV into a pandas DataFrame
data = StringIO(csv_string)
df = pd.read_csv(data)

print("Data Loaded Successfully:")
print(df.head())

Data Loaded Successfully:
   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  Complain  Satisfaction Score Card Type  \
0        101348.88       1         1                   2   DIA

## H2O for Model building

In [39]:
# Step 3: Start the H2O cluster
h2o.init()

# Step 4: Define significant features and the target column
significant_features = [
    'Gender', 'Geography', 'IsActiveMember', 'Age', 'CreditScore',
    'Balance', 'Satisfaction Score', 'Point Earned', 'Tenure',
    'EstimatedSalary', 'NumOfProducts', 'Card Type', 'HasCrCard','Complain'
]
target = 'Exited'

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,19 mins 32 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,"7 days, 20 hours and 35 minutes"
H2O_cluster_name:,H2O_from_python_unknownUser_ur1gyx
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.073 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


### Test Train split and preprocessing

In [40]:

# Step 5: Preprocess the data (ensure target column is categorical)
# Convert categorical columns using Label Encoding
categorical_columns = ['Gender', 'Geography', 'Card Type']
for col in categorical_columns:
    df[col] = df[col].astype('category')

# Split the data into training and test sets
X = df[significant_features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
# Combine training data for H2O
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

# Step 6: Convert pandas DataFrames to H2OFrames
h2o_data = h2o.H2OFrame(df_train)
h2o_test = h2o.H2OFrame(df_test)

# Ensure the target column is treated as categorical
h2o_data[target] = h2o_data[target].asfactor()
h2o_test[target] = h2o_test[target].asfactor()


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


### AutoMl Model


In [42]:
# Step 7: Initialize H2O AutoML
aml = H2OAutoML(max_models=20, seed=42, verbosity="info")


### Training


In [43]:

# Step 8: Train the AutoML model on the training set
aml.train(x=significant_features, y=target, training_frame=h2o_data)

AutoML progress: |
00:52:12.62: Project: AutoML_2_20241110_05212
00:52:12.62: 5-fold cross-validation will be used.
00:52:12.65: Setting stopping tolerance adaptively based on the training frame: 0.011180339887498949
00:52:12.65: Build control seed: 42
00:52:12.66: training frame: Frame key: AutoML_2_20241110_05212_training_py_6_sid_a487    cols: 15    rows: 8000  chunks: 1    size: 152556  checksum: -3697312703151767348
00:52:12.66: validation frame: NULL
00:52:12.66: leaderboard frame: NULL
00:52:12.66: blending frame: NULL
00:52:12.66: response column: Exited
00:52:12.66: fold column: null
00:52:12.66: weights column: null
00:52:12.66: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90w), lr_search (7g, 30w)]}, {GLM : [def_1 (1g, 10w)]}, {DRF : [def_1 (2g, 10w), XRT (3g, 10w)]}, {GBM : [def_5 (1g, 10w), def_2 (2g, 10w), def_3 (2g, 10w), def_4 (2g, 10w), def_1 (3g, 10w), grid_1 (4g, 60w), lr_annealing (7g, 10w)]}, {DeepLearning : [

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_2_AutoML_2_20241110_05212


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    88                 88                          75871                  7            7            7             27            99            63.8864

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 2.6974133366068792e-05
RMSE: 0.005193662808275947
LogLoss: 0.0007668779208770702
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7610674993420145
       0     1     Error    Rate
-----  ----  ----  -------  ------------
0      6355  0     0        (0.0/6355.0)
1      0     1645  0        (0.0/1645.0)
Total  6355  1645  0        (0.0/8000.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.761067     1         228
max f2                       0.761067     1         228
max f0point5                 0.761067     1         228
max accuracy                 0.761067     1         228
max precision                0.99981      1         0
max recall                   0.761067     1         228
max specificity              0.99981      1         0
max absolute_mcc             0.761067     1         228
max min_per_class_accuracy   0.761067     1         228
max mean_per_class_accuracy  0.761067     1         228
max tns                      0.99981      6355      0
max fns                      0.99981      1644      0
max fps                      7.07074e-05  6355      399
max tps                      0.761067     1645      228
max tnr                      0.99981      1         0
max fnr                      0.99981      0.999392  0
max fpr                      7.07074e-05  1         399
max tpr                      0.761067     1         228

Gains/Lift Table: Avg response rate: 20.56 %, avg score: 20.56 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.01                        0.999683           4.86322   4.86322            1                0.999728     1                           0.999728            0.0486322       0.0486322                  386.322   386.322            0.0486322
2        0.02                        0.999623           4.86322   4.86322            1                0.999652     1                           0.99969             0.0486322       0.0972644                  386.322   386.322            0.0972644
3        0.03                        0.999579           4.86322   4.86322            1                0.999604     1                           0.999661            0.0486322       0.145897                   386.322   386.322            0.145897
4        0.04                        0.999512           4.86322   4.86322            1                0.999541     1                           0.999631            0.0486322       0.194529                   386.322   386.322            0.194529
5        0.05                        0.999434           4.86322   4.86322            1                0.999477     1                           0.9996              0.0486322       0.243161          

In [44]:
# Step 9: View the leaderboard for the best models
lb = aml.leaderboard
print("H2O AutoML Leaderboard:")
print(lb.head(rows=lb.nrows))



H2O AutoML Leaderboard:
model_id                                                     auc     logloss     aucpr    mean_per_class_error       rmse         mse
GBM_2_AutoML_2_20241110_05212                           0.999599  0.00909586  0.998451              0.00161996  0.0384731  0.00148018
GBM_3_AutoML_2_20241110_05212                           0.999579  0.00923378  0.998412              0.00161996  0.0385214  0.0014839
GBM_5_AutoML_2_20241110_05212                           0.999571  0.00904848  0.998502              0.00161996  0.0385466  0.00148584
StackedEnsemble_BestOfFamily_1_AutoML_2_20241110_05212  0.999478  0.008662    0.998734              0.00161996  0.038334   0.0014695
DeepLearning_grid_3_AutoML_2_20241110_05212_model_1     0.999478  0.00953911  0.997465              0.00161996  0.0390039  0.0015213
GBM_4_AutoML_2_20241110_05212                           0.999455  0.00924641  0.998014              0.00161996  0.0385376  0.00148514
XGBoost_2_AutoML_2_20241110_05212        

### Evaluate on Test data

In [45]:
# Step 10: Evaluate the best model on the test set
best_model = aml.leader
perf = best_model.model_performance(h2o_test)
print("Model Performance:")
print(perf)


Model Performance:
ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.0009888358994221808
RMSE: 0.03144576123139939
LogLoss: 0.006614214068769838
Mean Per-Class Error: 0.0015834033989337362
AUC: 0.9995677308720912
AUCPR: 0.9989392509447373
Gini: 0.9991354617441823

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9329197367399474
       0     1    Error    Rate
-----  ----  ---  -------  ------------
0      1606  1    0.0006   (1.0/1607.0)
1      1     392  0.0025   (1.0/393.0)
Total  1607  393  0.001    (2.0/2000.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.93292      0.997455  206
max f2                       0.93292      0.997455  206
max f0point5                 0.93292      0.997455  206
max accuracy                 0.93292      0.999     206
max precision                0.999811     1         0
ma

### Store in Mojo format

In [46]:

# Step 11: Save the best model to a local file
model_path = best_model.download_mojo(path="/tmp")
model_filename = model_path.split('/')[-1]

print(f"Model saved locally as {model_filename}")

Model saved locally as GBM_2_AutoML_2_20241110_05212.zip


### Save it to S3


In [47]:

# Step 12: Upload the model to S3
s3.upload_file(
    Filename=model_path,
    Bucket=bucket,
    Key=f"h2o_models/{model_filename}"
)

print(f"Model uploaded to S3 at s3://{bucket}/h2o_models/{model_filename}")

Model uploaded to S3 at s3://newscatayerhs/h2o_models/GBM_2_AutoML_2_20241110_05212.zip


### Check if saved

In [48]:
response = s3.list_objects_v2(Bucket=bucket, Prefix="h2o_models/")
for obj in response.get('Contents', []):
    print(obj['Key'])


h2o_models/GBM_2_AutoML_2_20241110_05212.zip
